In [1]:
from PIL import Image
import hashlib

def dtrmine_range(a,b):      #determinning the difference b/w two adjacent pixels
    if(abs(a-b)<16):
        return 0
    elif(abs(a-b)>=16)and(abs(a-b)<32):
        return 1
    elif(abs(a-b)>=32):
        return 2

def sub(l,i,bindata,key,d1):
     #encoding the first pixel of the pair
    lst=[]
    lt=[]
    n11=l[i]
    n22=l[i+1]
    lst.append(format(l[i],'08b'))
    for k in range(8):
        lt.append(lst[0][k])
    
    for j in range(-key,0):
        if(len(bindata)==0):
            break
        else:
            lt[j]=bindata[0]
            bindata.pop(0)
    n1=0
    for j in range(7,-1,-1):
        n1=n1+int(lt[0])*pow(2,j)
        lt.pop(0)
       
    #encoding the second pixel of the pair
    lst=[]
    lst.append(format(l[i+1],'08b'))
    lt=[]
    for k in range(8):
        lt.append(lst[0][k])
    
    for j in range(-key,0):
        if(len(bindata)==0):
            break
        else:
            lt[j]=bindata[0]
            bindata.pop(0)   
    n2=0
    for j in range(7,-1,-1):
        n2=n2+int(lt[0])*pow(2,j)
        lt.pop(0)
    
    #readjustment
    N1=n1
    N2=n2
    if(n1>=n2):
        n1=n1-pow(2,key)
        if(n2+pow(2,key)<=255):
            n2=n2+pow(2,key)
    else:
        if(n1+pow(2,key)<=255):
            n1=n1+pow(2,key)
        n2=n2-pow(2,key)
     
    #choosing the best choice
    d2=dtrmine_range(n1,n2)
    #case 1
    if d1==0 and (d2==1 or d2==2):
        if n1>=n2:
            n111=N1
            n221=n2+pow(2,key)
            
            n112=n1-pow(2,key)
            n222=N2
        elif n1<n2:
            n111=n1+pow(2,key)
            n221=N2
            
            n112=N1
            n222=n2-pow(2,key)
            
        dev1=pow(abs(n11-n111),2)+pow(abs(n22-n221),2)
        dev2=pow(abs(n11-n112),2)+pow(abs(n22-n222),2)
        
        if(dev1>=dev2):
            n1=n112
            n2=n222
        elif(dev1<dev2):
            n1=n111
            n2=n221
    
    #case 2
    elif d1==1 and d2==0:
        if n1>=n2:
            n111=n1+pow(2,key)
            n221=N2
            
            n112=N1
            n222=n2-pow(2,key)
        elif n1<n2:
            n111=N1
            n221=n2+pow(2,key)
            
            n112=n1-pow(2,key)
            n222=N2
            
        dev1=pow(abs(n11-n111),2)+pow(abs(n22-n221),2)
        dev2=pow(abs(n11-n112),2)+pow(abs(n22-n222),2)
        
        if(dev1>=dev2):
            n1=n112
            n2=n222
        elif(dev1<dev2):
            n1=n111
            n2=n221
    #case3        
    elif d1==1 and d2==2:
        if n1>=n2:
            n111=n1-pow(2,key)
            n221=N2
            
            n112=N1
            n222=n2+pow(2,key)
        elif n1<n2:
            n111=N1
            n221=n2-pow(2,key)
            
            n112=n1+pow(2,key)
            n222=N2
            
        dev1=pow(abs(n11-n111),2)+pow(abs(n22-n221),2)
        dev2=pow(abs(n11-n112),2)+pow(abs(n22-n222),2)
        
        if(dev1>=dev2):
            n1=n112
            n2=n222
        elif(dev1<dev2):
            n1=n111
            n2=n221
       
    #case 4
    elif d1==2 and (d2==0 or d2==1):
        if n1>=n2:
            n111=n1+pow(2,key)
            n221=N2
            
            n112=N1
            n222=n2-pow(2,key)
        else:
            n111=n1-pow(2,key)
            n221=N2
            
            n112=N1
            n222=n2+pow(2,key)
         
        dev1=pow(abs(n11-n111),2)+pow(abs(n22-n221),2)
        dev2=pow(abs(n11-n112),2)+pow(abs(n22-n222),2)
        
        if(dev1>=dev2):
            n1=n112
            n2=n222
        elif(dev1<dev2):
            n1=n111
            n2=n221
       
        
    
    return n1,n2,bindata 

 
def getlen(imge):
    w=imge.size[0]
    h=imge.size[1]
    return w,h


def pvd_encode(imge,data):
    l=list(imge.getdata())
    w,h=getlen(imge)
    
    #generating binary code of the data to be encoded
    newd=[]
    bindata=[]
    for i in data:
        newd.append(format(ord(i),'08b'))
    
    for g in range(len(newd)):
        for j in range(8):
            bindata.append(newd[g][j])      
    
    #encoding the message
    (x,y)=(0,0)
    for i in range(0,w*h,2):
        if len(bindata)!=0:
            d1=dtrmine_range(l[i],l[i+1])
            if d1==2:
                n1,n2,bindata=sub(l,i,bindata,5,d1)
            elif d1==1:
                n1,n2,bindata=sub(l,i,bindata,4,d1)
            elif d1==0:
                n1,n2,bindata=sub(l,i,bindata,3,d1)
            
            imge.putpixel((x,y),n1)
            if(x<w-1) and (y<=h-2):
                x=x+1
            elif(x==w-1) and (y<h-2):
                x=0
                y=y+1    
            elif(x==w-1) and (y==h-2):
                raise ValueError("image size is not sufficient to encode the data")
            
            imge.putpixel((x,y),n2) 
            if(x<w-1) and (y<=h-2):
                x=x+1
            elif(x==w-1) and (y<h-2):
                x=0
                y=y+1    
            elif(x==w-1) and (y==h-2):
                raise ValueError("image size is not sufficient to encode the data")
        else:
            break
            
    #encoding the length of the message using normal lsb steganography
    length=len(data)
    len_data=[]
    len_data.append(format(length,'08b'))
    for f in range(-8,0):
        if(l[f]%2==0) and (len_data[0][8+f]=='1'):
            l[f]=l[f]+1
        elif(l[f]%2==1) and (len_data[0][8+f]=='0'):
            l[f]=l[f]-1    
        imge.putpixel((w+f,h-1),l[f])
    
            
    imge.save('pvd_steg.png')           
  



In [2]:
def generate_hash_code():
    img=Image.open('macao.png')
    l=list(img.getdata())
    str1=''
    for i in range(len(l)):
        str1+="%s"%l[i]
        
    result=hashlib.sha256(str1.encode())
    
    return result.hexdigest()

In [3]:
def encode(data):
    img=Image.open('macao.png')
    #data=generate_hash_code()
    
    if(len(data)==0):
        raise ValueError("Data is empty")  
    newimg=img 
    pvd_encode(newimg,data)
    
    
    #new_img_name=input("enter the name of the new image(with extension):")
    #newimg.save(new_img_name,str(new_img_name.split(".")[1].upper()))          

In [5]:
data = 'aritra raut'
encode(data)